In [1]:
import ee
import os
import math
from datetime import datetime
from src.utils import ee_treatments
from src.utils import runs
from src.utils.check_exists import check_exists
from src.utils.yml_params import get_export_params
yml_file = os.path.join(os.getcwd(),'config.yml')
CRS,EXPORT_SCALE = get_export_params(yml_file)

ee.Initialize(project='mas-gee')
%load_ext autoreload
%autoreload 2

# In development phase
tag = 'ee_treatments_production'
ee.data.setWorkloadTag(tag)

## Million Acres Strategy - Treatment Simulator
#### Creates Randomized Treatment Scenario Landscapes - exports as ee.Images
#### This Notebook is for Run IDs: 
|RunID | CA_REGION | INTENSITY_ID | PRIORITY |
|------|-----------|--------------|----------|
| 19  | South Coast | 500k | WUI |
| 22  | South Coast | 1m   | WUI |
| 25  | South Coast | 2m   | WUI |
| 28 | Central Coast   | 500k | WUI |
| 31 | Central Coast   | 1m   | WUI |
| 34 | Central Coast   | 2m   | WUI |


Run Settings: Change `RUNID` below to setup the analysis and output file names

In [2]:
RUNID = 'RunID22' 

Auto-calculate the necessary information for the Scenario

Double-Check the Output Calcs are Correct against the [Spreadsheet](https://docs.google.com/spreadsheets/d/1Gnl6SO5kOkj4Ne1JdXzW4bp03824Zb1Cp_2HVuCY3Lk/edit#gid=0
)

In [3]:
# string identifiers for output file
CA_REGION = runs.key[RUNID]['RegionID'] # NC = North Coast; SN = Sierra Nevada
INTENSITY_ID = runs.key[RUNID]['IntensityID'] 
PRIORITY = runs.key[RUNID]['PriorityID'] 

# construct output path
today_string = datetime.utcnow().strftime("%Y-%m-%d").replace("-", "")
output_image = f"projects/mas-gee/assets/treatment_scenarios/{RUNID}/{CA_REGION}_{PRIORITY}_{INTENSITY_ID}_{today_string}_test"
print(f'Output EE Image Path: {output_image}')

# treatment size is static and set in runs.py
treatment_size_ac = runs.treatment_size_ac 
TREATMENT_SIZE = runs.treatment_size_sqm 
print('TREATMENT SIZE (m^2):',TREATMENT_SIZE)
print('TREATMENT SIZE (Ac):',treatment_size_ac)
RADIUS = math.sqrt(TREATMENT_SIZE)/2  # in meters, rough square radius is A = (side/2) solve for side (side = sqrt(A) )

sqm_to_ac_x = 0.000247105

# pull prescribed treatment area in both units, we use sqm for analysis and acreage for reporting
total_treated_area = runs.key[RUNID]['sqmPerYear']
print(f"Area to Treat Per Year (m^2): {total_treated_area}")
total_treated_area_ac = runs.key[RUNID]['acreagePerYear']
print(f"Area to Treat Per Year (Ac): {total_treated_area_ac}")

# we run the treatment generator as 5-year interval outputs (4 run intervals covering the 20-year period)
# so we actually need to total_treated_area*5 to the ee_treatments() generator
area_per_iteration = int(round(total_treated_area*5))
print(f'Area to treat per 5-year interval (m^2): {area_per_iteration}')
area_per_iteration_ac = int(round(area_per_iteration*sqm_to_ac_x))
print(f'Area to treat per 5-year interval (Ac): {area_per_iteration_ac}')

Output EE Image Path: projects/mas-gee/assets/treatment_scenarios/RunID19/SC_WUI_500k_20230922_test
TREATMENT SIZE (m^2): 404685.64224
TREATMENT SIZE (Ac): 100
Area to Treat Per Year (m^2): 323748513.792
Area to Treat Per Year (Ac): 80000
Area to treat per 5-year interval (m^2): 1618742569
Area to treat per 5-year interval (Ac): 399999


Input Files

In [8]:
# Sierra Nevada(SN), South Coast(SC), Central Coast(CC), North Coast(NC)
all_hucs = ee.FeatureCollection("projects/mas-gee/assets/TxHucScCc")
# property names differ between Wui, BP, and RFFc
propSwName = 'WpropSw'
propGdName = 'WpropGd'

# Treatable Vegetation Constraint Layer
# For reference Herb (1), Shrub (2), Hardwood (3), softwood (4), non treatable veg is masked
treatable_veg_layer = ee.Image("projects/mas-gee/assets/HerbShrubHardSoftImage")
# Softwoods
veg_constraint_Sw = treatable_veg_layer.eq(4) # only considering softwood
# Road Grids
veg_constraint_Gd = ee.Image(runs.key[RUNID]['RoadGrids'])

# print(veg_constraint_Gd.bandNames().getInfo())
# print(veg_constraint_Sw.bandNames().getInfo())


['constant']
['remapped']


Need to proportionally allocate treated area between softwood veg and road-adjacent treatment cells

There is a different softwood/road grid area proportion split per priority scheme (WUI,BP,RFFC) and percentile Group within those

It is likely going to be easier to pull that specific pair of proportions out ahead of time from the HUC FC or hard-code it here...

Pay attention to `overshoot` value passed in `ee_treatments()` call. It is your dial. 

In [22]:
PT_OVERSHOOT_Sw = 2.4
PT_OVERSHOOT_Gd = 2.2

WUI Prioritization. 

see [Treatment Allocation Runs sheet](https://docs.google.com/spreadsheets/d/1Gnl6SO5kOkj4Ne1JdXzW4bp03824Zb1Cp_2HVuCY3Lk/edit#gid=0)

* 1-25th percentile HUC group in Year Interval 1 and 4
* 25th-50th percentile HUC group in Year Interval 2
* 50-75th percentile HUC group in Year Interval 3.

In [65]:
year_interval_ids = ee.List(['Y1to5',
                             'Y6to10',
                             'Y11to15',
                             'Y16-20'
                             ]
                             )
ranked_huc_filters = [ee.Filter.eq('TxWPrct', 25),
               ee.Filter.eq('TxWPrct', 50),
               ee.Filter.eq('TxWPrct', 75),
               ee.Filter.eq('TxWPrct', 25)
               ]
# zipping ee.Lists is a good way to parallelize complicated things in GEE with .map()
zipped = year_interval_ids.zip(ranked_huc_filters)
# print(zipped.getInfo())

# even more info needs to be passed into the map so we'll create n sub-lists
trt_type_Sw = ee.List.repeat(propSwName,4)
trt_type_Gd = ee.List.repeat(propGdName,4)
pt_overshoots_Sw = ee.List.repeat(PT_OVERSHOOT_Sw,4)
pt_overshoots_Gd = ee.List.repeat(PT_OVERSHOOT_Gd,4)
n_items = ee.List.sequence(0,3)

def sub_lists_Sw(i):
    # at each index position return all three elements from the same position
    yid = ee.List(year_interval_ids).get(i)
    filt = ee.List(ranked_huc_filters).get(i)
    trt = ee.List(trt_type_Sw).get(i)
    pto = ee.List(pt_overshoots_Sw).get(i)
    veg_constraint_layer = veg_constraint_Sw
    return ee.List([yid,filt,trt,pto,veg_constraint_layer])

def sub_lists_Gd(i):
    # at each index position return all three elements from the same position
    yid = ee.List(year_interval_ids).get(i)
    filt = ee.List(ranked_huc_filters).get(i)
    trt = ee.List(trt_type_Gd).get(i)
    pto = ee.List(pt_overshoots_Gd).get(i)
    veg_constraint_layer = veg_constraint_Gd
    return ee.List([yid,filt,trt,pto,veg_constraint_layer])

SW_PARAMS_LIST = n_items.map(sub_lists_Sw)
GD_PARAMS_LIST = n_items.map(sub_lists_Gd)
# print(ee.List(ee.List(SW_PARAMS_LIST).get(0)).get(4).getInfo())
# print(ee.List(ee.List(GD_PARAMS_LIST).get(0)).get(4).getInfo())



{'type': 'Image', 'bands': [{'id': 'remapped', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1}, 'dimensions': [915, 1060], 'crs': 'EPSG:3310', 'crs_transform': [1000.0000000000001, 0, -373980, 0, -1000.0000000000001, 455280]}]}
{'type': 'Image', 'bands': [{'id': 'constant', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [445, 311], 'crs': 'EPSG:3310', 'crs_transform': [1000, 0, -62000, 0, -1000, -295000]}], 'version': 1695396870268674, 'id': 'projects/mas-gee/assets/scTxGridRoadImage', 'properties': {'system:footprint': {'type': 'LinearRing', 'coordinates': [[-120.68828933725352, 35.35649291861447], [-120.68836742110692, 35.355357707099216], [-120.6644671044239, 32.560310297632476], [-120.66009797592056, 32.55619276296378], [-120.65638160931445, 32.5516427749698], [-120.65461943623058, 32.55136977354716], [-118.29411750191537, 32.54024672235406], [-115.93661433098698, 32.47923912369448], [-115.93152519972557, 32.48

In [66]:
def treat_by_year_huc(l):
    """
    this is to be map()ed across zipped (year intervals and percentile huc filters) to parallelize use of ee_treatments()
        result is one ee.Image of treatment areas per yearInterval/HUC Group element in zipped     
    """
    
    # pulling parameters for the run out of the given sub-list (either SW_PARAMS_LIST or GD_PARAMS_LIST)
    zipped_list = ee.List(l)
    ranks_filter = ee.Filter(zipped_list.get(1))
    trt_prop_name = ee.String(zipped_list.get(2))
    pt_overshoot = ee.Number(zipped_list.get(3))
    veg_constraint = zipped_list.get(4)
    # which PARAMS_LIST are we in? 
    PARAMS_LIST = ee.Algorithms.If(ee.String(trt_prop_name).match(propSwName),SW_PARAMS_LIST,GD_PARAMS_LIST)
    # pixel value of resulting image represents iteration number (e.g. 1 = years 1-5, e = years 6-10,etc)
    pixel_value_iteration = (ee.Number(
       ee.List(PARAMS_LIST) ## <---- MUST CHANGE THIS UPON RUNNING THIS CELL - couldn't figure out how to make this change
       .indexOf(l))
       .add(1))

    # Get HUC group
    huc_group = (all_hucs.filter(ee.Filter.eq('RRK_Rgn', runs.key[RUNID]['Region Filter Name']))
                .filter(ranks_filter)
                # .limit(2) # for testing, reduce compute for development
                )
    
    # Additional CC/SC steps ---------
    # we are running ee_treatments() twice and returning their individual results. 
    # The three inputs that differ per run are:
    # 1. the veg constraint layer (Softwoods mask or Road Grids mask)
    # 2. prescription area (determined individually for Softwoods versus Road Grids by area proportion per HUC percentile Group
    # 3. and PT_OVERSHOOT value for each run
    
    # area_per_iteration is the total prescribed area we're placing
    # area_per_iteration needs to be proportionally allocated to softwood area and road grid areas
    propWeight = ee.Number(ee.FeatureCollection(huc_group).first().get(trt_prop_name))
    # propSw = ee.Number(ee.FeatureCollection(huc_group).first().get(propSwName))
    # propGd = ee.Number(ee.FeatureCollection(huc_group).first().get(propGdName))
    
    prescription_area = ee.Number(area_per_iteration).multiply(propWeight)
    # area_per_iteration_Sw = ee.Number(area_per_iteration).multiply(propSw) # this is passed to ee_treatments()    
    # area_per_iteration_Gd = ee.Number(area_per_iteration).multiply(propSw)
    
    
    # this is for reshuffling the random treatment placements, may be useful for re-treatments of the same huc groups
    # we would pass a unique number to 'seed' arg in ee_treatments() each time 
    # if you don't need to reshuffle the random seed each run, don't define 'seed' arg and function will use its default seed every time
    seeds = ee.List([10,20,30,40])
    seed = ee.Number(ee.List(seeds).get(ee.List(zipped).indexOf(l)))
    
    treated_area_img, properties = ee_treatments.ee_treatments(hucs=huc_group,
                                                  prescription=prescription_area, # area_per_iteration_testing , reduce compute for testing
                                                  unit_size=TREATMENT_SIZE,
                                                  radius=RADIUS,
                                                  pixel_value=pixel_value_iteration,
                                                  constraint_layer=veg_constraint,
                                                  # seed=seed # passing seed defined as diff number for every ee_treatment() run to randomize re-treatment groups
                                                  ptOvershoot=pt_overshoot # we dial this in per scenario to get close to a 1:1 of Needed:Generated
                                                  )
    treated_area_img = treated_area_img.set('trt_type',trt_prop_name)
    
    return treated_area_img, properties

def remap_keys(d):
  """
  solves a QA property problem for ee_treatments(): we have duplicate key names in the properties dict 
  when multiple ee_treatments() results are combined together.
  So we create a unique identifier from 'yearInterval' and remove 'yearInterval' as a property itself.
  example: 'PropertyName': PropertyValue becomes 'yearInterval1PropertyName': PropertyValue
  """
  d = ee.Dictionary(d)
  yearInterval = (ee.String('yearInterval')
  .cat(ee.Number(ee.Dictionary(d).get('yearInterval')).format()))
  keys = d.keys().slice(0,7) # depends on yearInterval property being in index position 7 of property list returned by ee_treatments()
  values = d.values().slice(0,7)
  remapped_keys = keys.map(lambda s: ee.String(yearInterval).cat(s))
  return ee.Dictionary.fromLists(remapped_keys,values)

def combine_dicts(d):
   d = ee.Dictionary(d)
   return d.keys().zip(d.values())

Create treated area ee.Images for every year interval on its assigned group of HUCs

In [68]:
# We use .map() to parallelize computation of treatment area images across the four 5-year intervals, 
# treating equal amount of area within each percentile ranking list, returning 4 treatment ee.Images, 
# then mosaicking them into one ee.Image for export

# returns ee.List((ee.Image1,ee.Dictionary1),(ee.Image2,ee.Dictionary2),(etc))
results_Sw = ee.List(SW_PARAMS_LIST).map(treat_by_year_huc)
results_Gd = ee.List(GD_PARAMS_LIST).map(treat_by_year_huc)
# print(results_Sw.getInfo())

# parse the images and properties out of the (image,properties) tuples by .map()ing over results
images_Sw = results_Sw.map(lambda t: ee.List(t).get(0)) 
properties_Sw = results_Sw.map(lambda t: ee.List(t).get(1))

images_Gd = results_Gd.map(lambda t: ee.List(t).get(0)) 
properties_Gd = results_Gd.map(lambda t: ee.List(t).get(1))

# print(images_Sw.getInfo())
# print(images_Gd.getInfo())
# print(properties_Sw.getInfo())
# print(properties_Gd.getInfo())

# remap the keys of the properties so they are unique per ee_treatments() image run, then combine into one properties dict
all_properties_Sw = properties_Sw.map(remap_keys).map(combine_dicts).flatten()
all_properties_Gd = properties_Gd.map(remap_keys).map(combine_dicts).flatten()
# print(all_properties_Sw.getInfo())

# mosaic all ee_treatment() images together and set the fixed properties 
output_Sw = ee.Image(ee.ImageCollection.fromImages(images_Sw).mosaic()).set(all_properties_Sw)
output_Gd = ee.Image(ee.ImageCollection.fromImages(images_Gd).mosaic()).set(all_properties_Gd)

# export
desc = f'{RUNID}_{os.path.basename(output_image)}'
folder = os.path.dirname(output_image)
if check_exists(folder):
    print(f'making new folder: {folder}')
    os.popen(f'earthengine create folder {folder}').read()
    
# ee_treatments.export_img(output_Sw,desc+'Sw',output_image+'Sw',all_hucs.geometry(),30,'EPSG:5070')
ee_treatments.export_img(output_Gd,desc+'Gd',output_image+'Gd',all_hucs.geometry(),30,'EPSG:5070')


Export Started for projects/mas-gee/assets/treatment_scenarios/RunID19/SC_WUI_500k_20230922_testGd


In [16]:
# We use .map() to parallelize computation of treatment area images across the four 5-year intervals, 
# treating equal amount of area within each percentile ranking list, returning 4 treatment ee.Images, 
# then mosaicking them into one ee.Image for export

# # returns ee.List((ee.Image1,ee.Dictionary1),(ee.Image2,ee.Dictionary2),(etc))
# results = ee.List(zipped).map(treat_by_year_huc)
# # print(results.getInfo())

# # parse the images and properties out of the (image,properties) tuples by .map()ing over results
# images = results.map(lambda t: ee.List(t).get(0)) 
# properties = results.map(lambda t: ee.List(t).get(1))

# # remap the keys of the properties so they are unique per ee_treatments() image run, then combine into one properties dict
# all_properties = properties.map(remap_keys).map(combine_dicts).flatten()
# # print(all_properties.getInfo())

# # mosaic all ee_treatment() images together and set the fixed properties 
# output = ee.Image(ee.ImageCollection.fromImages(images).mosaic()).set(all_properties)

# # export
# desc = f'{RUNID}_{os.path.basename(output_image)}'

# folder = os.path.dirname(output_image)
# if check_exists(folder):
#     print(f'making new folder: {folder}')
#     os.popen(f'earthengine create folder {folder}').read()
# ee_treatments.export_img(output,desc,output_image,all_hucs.geometry(),30,'EPSG:5070')

AttributeError: 'ApiFunction' object has no attribute 'name'

Pulling out image and properties from both ee_treatments() runs now instead of just the one

Separate function for making treated area Image from Road Grid Cells per Percentile HUC Group

In [19]:
# def treat_by_year_huc_Gd_cells(l):
#     """
#     this is to be map()ed across zipped (year intervals and percentile huc filters) to parallelize use of ee_treatments()
#         result is one ee.Image of treatment areas per yearInterval/HUC Group element in zipped     
#     """
#     zipped_list = ee.List(l)
#     ranks_filter = ee.Filter(zipped_list.get(1))
    
#     # pixel value of resulting image represents iteration number (e.g. 1 = years 1-5, e = years 6-10,etc)
#     pixel_value_iteration = ee.Number(ee.List(zipped).indexOf(l)).add(1)

#     # Get HUC group
#     huc_group = (all_hucs.filter(ee.Filter.eq('RRK_Rgn', runs.key[RUNID]['Region Filter Name']))
#                 .filter(ranks_filter)
#                 # .limit(2) # for testing, reduce compute for development
#                 )
#     propGd = ee.Number(ee.FeatureCollection(huc_group).first().get(propGdName))
#     # we are creating a simple polygon->raster result from the road grid cell polygons based on the number of road grid cells we need   
#     area_per_iteration_ac_Gd = ee.Number(area_per_iteration_ac).multiply(propGd) 
#     prescribed_Gd_cell_count = area_per_iteration_ac_Gd.divide(100).round() # how many 100ac road grid cells we need for prescription
#     Gd_cells_filtered = Gd_cells.filterBounds(huc_group).randomColumn().limit(prescribed_Gd_cell_count,'random',True) # randomly select the amount we need
#     Gd_cells_img = ee.Image(0).paint(Gd_cells_filtered,pixel_value_iteration) # road grid cells burned in as pixel_value_iteration (same as ee_treatments())
#     Gd_cells_properties = ee.Dictionary({'area_per_iteration_ac_gd':area_per_iteration_ac_Gd,
#                                          'prescribed_Gd_cell_count':prescribed_Gd_cell_count,
#                                          'yearInterval':pixel_value_iteration})
#     return Gd_cells_img.set(Gd_cells_properties).rename('Tx'), Gd_cells_properties

# # had to redefine this cause of idx position of 'yearInterval' has to change
# def remap_keys(d):
#   """
#   solves a QA property problem for ee_treatments(): we have duplicate key names in the properties dict 
#   when multiple ee_treatments() results are combined together.
#   So we create a unique identifier from 'yearInterval' and remove 'yearInterval' as a property itself.
#   example: 'PropertyName': PropertyValue becomes 'yearInterval1PropertyName': PropertyValue
#   """
#   d = ee.Dictionary(d)
#   yearInterval = (ee.String('yearInterval')
#   .cat(ee.Number(ee.Dictionary(d).get('yearInterval')).format()))
#   keys = d.keys().slice(0,3) # depends on yearInterval property being in certain index position of property list returned by ee_treatments()
#   values = d.values().slice(0,3)
#   remapped_keys = keys.map(lambda s: ee.String(yearInterval).cat(s))
#   return ee.Dictionary.fromLists(remapped_keys,values)


Export Road Grid Cell Image


In [24]:

# # retrive list of treated images
# results = ee.List(zipped).map(treat_by_year_huc_Gd_cells)
# # print(results.getInfo())

# # parse the images and properties out of the (image,properties) tuples by .map()ing over results
# images = results.map(lambda t: ee.List(t).get(0)) 
# properties = results.map(lambda t: ee.List(t).get(1))
# # print(properties.getInfo())
# # remap the keys of the properties so they are unique per ee_treatments() image run, then combine into one properties dict
# all_properties = properties.map(remap_keys).map(combine_dicts).flatten()
# # print(all_properties.getInfo())

# #mosaic them 
# out_img = ee.Image(ee.ImageCollection.fromImages(images).mosaic()).set(all_properties)
# out_Gd_path = f"{output_image}_Road_Grids"
# # export
# desc_Gd = f'{RUNID}_{os.path.basename(output_image)}_road_grid'

# folder = os.path.dirname(output_image)
# if check_exists(folder):
#     print(f'making new folder: {folder}')
#     os.popen(f'earthengine create folder {folder}').read()
# ee_treatments.export_img(out_img,desc_Gd,out_Gd_path,all_hucs.geometry(),30,'EPSG:5070')


['yearInterval1area_per_iteration_ac_gd', 340081.7497935, 'yearInterval1prescribed_Gd_cell_count', 3401, 'yearInterval1yearInterval', 1, 'yearInterval2area_per_iteration_ac_gd', 292140.7896462, 'yearInterval2prescribed_Gd_cell_count', 2921, 'yearInterval2yearInterval', 2, 'yearInterval3area_per_iteration_ac_gd', 233225.17693560003, 'yearInterval3prescribed_Gd_cell_count', 2332, 'yearInterval3yearInterval', 3, 'yearInterval4area_per_iteration_ac_gd', 340081.7497935, 'yearInterval4prescribed_Gd_cell_count', 3401, 'yearInterval4yearInterval', 4]
Export Started for projects/mas-gee/assets/treatment_scenarios/RunID19/SC_WUI_500k_20230921_test_Road_Grids
